In [1]:
import numpy as np
x=np.load('/../data/legal_NLP/legalNLP_data/final_embd_files/sent2vec_embd_files/final_sv_mincount5_dim200_Vsize250000_Ngram2/train_embd_final_sv_mincount5_dim200_Vsize250000_Ngram2.npy',allow_pickle='true')
x_train_data=x[:,0]
y_train=x[:,1]
y_train=y_train.astype('int')
x_train=np.zeros([32305,200])
for i in range(0,32305):
    b=np.zeros([len(x_train_data[i]),200])
    for j in range(0,len(x_train_data[i])):
        b[j,:]=x_train_data[i][j]
    b=np.sum(b,axis=0)
    x_train[i,:]=b/len(x_train_data[i])


In [2]:
print(x_train.shape)

(32305, 200)


In [3]:
y=np.load('/../data/legal_NLP/legalNLP_data/final_embd_files/sent2vec_embd_files/final_sv_mincount5_dim200_Vsize250000_Ngram2/test_embd_final_sv_mincount5_dim200_Vsize250000_Ngram2.npy',allow_pickle='true')
x_test_data=y[:,0]
y_test=y[:,1]
y_test=y_test.astype('int')
x_test=np.zeros([1517,200])
for i in range(0,1517):
    b=np.zeros([len(x_test_data[i]),200])
    for j in range(0,len(x_test_data[i])):
        b[j,:]=x_test_data[i][j]
    b=np.sum(b,axis=0)
    x_test[i,:]=b/len(x_test_data[i])

In [4]:
z=np.load('/../data/legal_NLP/legalNLP_data/final_embd_files/sent2vec_embd_files/final_sv_mincount5_dim200_Vsize250000_Ngram2/validation_embd_final_sv_mincount5_dim200_Vsize250000_Ngram2.npy',allow_pickle='true')
x_dev_data=z[:,0]
y_dev=z[:,1]
y_dev=y_dev.astype('int')
x_dev=np.zeros([994,200])
for i in range(0,994):
    b=np.zeros([len(x_dev_data[i]),200])
    for j in range(0,len(x_dev_data[i])):
        b[j,:]=x_dev_data[i][j]
    b=np.sum(b,axis=0)
    x_dev[i,:]=b/len(x_dev_data[i])


In [5]:
print(x_train)
print(y_test.shape)
print(y_dev.shape)

[[ 0.23465343 -0.12870004 -0.12489287 ...  0.05784892  0.04879328
   0.18257203]
 [ 0.0843488  -0.14360571 -0.19869974 ...  0.0475443   0.11492499
  -0.06642166]
 [ 0.15217664 -0.07390564 -0.10769357 ... -0.00774169  0.01975319
  -0.06602175]
 ...
 [-0.10669029 -0.0455214  -0.0211758  ... -0.00930675  0.11525618
   0.043224  ]
 [ 0.03532715 -0.06977894 -0.06398666 ...  0.03205929 -0.02879568
   0.00917467]
 [ 0.08008892 -0.17394208 -0.11530315 ...  0.0074436   0.0226157
  -0.05872493]]
(1517,)
(994,)


In [6]:
y_train1=np.zeros([32305,1])
for i in range(0,32305):
    y_train1[i][0]=y_train[i]

In [7]:
y_test1=np.zeros([1517,1])
for i in range(0,1517):
    y_test1[i][0]=y_test[i]

In [8]:
y_dev1=np.zeros([994,1])
for i in range(0,994):
    y_dev1[i][0]=y_dev[i]

In [9]:
print(x_train)

[[ 0.23465343 -0.12870004 -0.12489287 ...  0.05784892  0.04879328
   0.18257203]
 [ 0.0843488  -0.14360571 -0.19869974 ...  0.0475443   0.11492499
  -0.06642166]
 [ 0.15217664 -0.07390564 -0.10769357 ... -0.00774169  0.01975319
  -0.06602175]
 ...
 [-0.10669029 -0.0455214  -0.0211758  ... -0.00930675  0.11525618
   0.043224  ]
 [ 0.03532715 -0.06977894 -0.06398666 ...  0.03205929 -0.02879568
   0.00917467]
 [ 0.08008892 -0.17394208 -0.11530315 ...  0.0074436   0.0226157
  -0.05872493]]


In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

def metrics_calculator(preds, test_labels):
    cm = confusion_matrix(test_labels, preds)
    TP = []
    FP = []
    FN = []
    for i in range(0,2):
        summ = 0
        for j in range(0,2):
            if(i!=j):
                summ=summ+cm[i][j]

        FN.append(summ)
    for i in range(0,2):
        summ = 0
        for j in range(0,2):
            if(i!=j):
                summ=summ+cm[j][i]

        FP.append(summ)
    for i in range(0,2):
        TP.append(cm[i][i])
    precision = []
    recall = []
    for i in range(0,2):
        precision.append(TP[i]/(TP[i] + FP[i]))
        recall.append(TP[i]/(TP[i] + FN[i]))

    macro_precision = sum(precision)/2
    macro_recall = sum(recall)/2
    micro_precision = sum(TP)/(sum(TP) + sum(FP))
    micro_recall = sum(TP)/(sum(TP) + sum(FN))
    micro_f1 = (2*micro_precision*micro_recall)/(micro_precision + micro_recall)
    macro_f1 = (2*macro_precision*macro_recall)/(macro_precision + macro_recall)
    return macro_precision, macro_recall, macro_f1, micro_precision, micro_recall, micro_f1
def RF_scores(train_avg, dev_avg, test_avg, train_labels, dev_labels, test_labels):
    f = open("RF_results.txt", "w+")
    f.write("Varying the n_estimators from 50 to 500\n\n")
    for n_est in range(50,500,50):
        clf=RandomForestClassifier(n_estimators=n_est)
        clf.fit(train_avg,train_labels)
        d_preds = clf.predict(dev_avg)
        Heading = "For n_estimators: " + str(n_est) + "\n"
        d_res = "Accuracy -> " + str(accuracy_score(d_preds, dev_labels)*100) + "\n"
        macro_precision, macro_recall, macro_f1, micro_precision, micro_recall, micro_f1 = metrics_calculator(d_preds, dev_labels)
        d_metrics = "Micros : " + str(micro_precision) + " " + str(micro_recall) + " " + str(micro_f1) + " Macros: " + str(macro_precision) + " " + str(macro_recall) + " " + str(macro_f1) + "\n"
        f.write(Heading + "Dev set:\n"+ d_res + d_metrics)
        
        t_preds = clf.predict(test_avg)
        t_res = "Accuracy -> " + str(accuracy_score(t_preds, test_labels)*100) + "\n"
        macro_precision, macro_recall, macro_f1, micro_precision, micro_recall, micro_f1 = metrics_calculator(t_preds, test_labels)
        t_metrics = "Micros : " + str(micro_precision) + " " + str(micro_recall) + " " + str(micro_f1) + " Macros: " + str(macro_precision) + " " + str(macro_recall) + " " + str(macro_f1) + "\n"
        f.write("Test set:\n"+ t_res + t_metrics + "\n\n")
        
    f.close()
def LR_scores(train_avg, dev_avg, test_avg, train_labels, dev_labels, test_labels):
    f = open("LR_results.txt", "w+")
    f.write("Varying the max_iters from 50 to 500\n\n")
    for it in range(50,500,50):
        LR = LogisticRegression(C=1, max_iter =it)
        LR.fit(train_avg, train_labels)
        d_preds = LR.predict(dev_avg)
        Heading = "For max_iters: " + str(it) + "\n"
        d_res = "Accuracy -> " + str(accuracy_score(d_preds, dev_labels)*100) + "\n"
        macro_precision, macro_recall, macro_f1, micro_precision, micro_recall, micro_f1 = metrics_calculator(d_preds, dev_labels)
        d_metrics = "Micros : " + str(micro_precision) + " " + str(micro_recall) + " " + str(micro_f1) + " Macros: " + str(macro_precision) + " " + str(macro_recall) + " " + str(macro_f1) + "\n"
        f.write(Heading + "Dev set:\n"+ d_res + d_metrics)
        
        t_preds = LR.predict(test_avg)
        t_res = "Accuracy -> " + str(accuracy_score(t_preds, test_labels)*100) + "\n"
        macro_precision, macro_recall, macro_f1, micro_precision, micro_recall, micro_f1 = metrics_calculator(t_preds, test_labels)
        t_metrics = "Micros : " + str(micro_precision) + " " + str(micro_recall) + " " + str(micro_f1) + " Macros: " + str(macro_precision) + " " + str(macro_recall) + " " + str(macro_f1) + "\n"
        f.write("Test set:\n"+ t_res + t_metrics + "\n\n")
        
    f.close()
def SVM_scores(train_avg, dev_avg, test_avg, train_labels, dev_labels, test_labels):
    f = open("SVM_results.txt", "w+")
    f.write("Varying the kernels: \n\n")
    kers = ["linear", "poly", "rbf"]
    for k in kers:
        print("Running for {0}".format(k))
        SVM = svm.SVC(C=1, kernel=k)
        SVM.fit(train_avg, train_labels)
        d_preds = SVM.predict(dev_avg)
        Heading = "For kernel: " + k + "\n"
        d_res = "Accuracy -> " + str(accuracy_score(d_preds, dev_labels)*100) + "\n"
        macro_precision, macro_recall, macro_f1, micro_precision, micro_recall, micro_f1 = metrics_calculator(d_preds, dev_labels)
        d_metrics = "Micros : " + str(micro_precision) + " " + str(micro_recall) + " " + str(micro_f1) + " Macros: " + str(macro_precision) + " " + str(macro_recall) + " " + str(macro_f1) + "\n"
        f.write(Heading + "Dev set:\n"+ d_res + d_metrics)
        print(Heading + "Dev set:\n"+ d_res + d_metrics)

        t_preds = SVM.predict(test_avg)
        t_res = "Accuracy -> " + str(accuracy_score(t_preds, test_labels)*100) + "\n"
        macro_precision, macro_recall, macro_f1, micro_precision, micro_recall, micro_f1 = metrics_calculator(t_preds, test_labels)
        t_metrics = "Micros : " + str(micro_precision) + " " + str(micro_recall) + " " + str(micro_f1) + " Macros: " + str(macro_precision) + " " + str(macro_recall) + " " + str(macro_f1) + "\n"
        f.write("Test set:\n"+ t_res + t_metrics + "\n\n")
        print("Test set:\n"+ t_res + t_metrics + "\n\n")
    f.close()

In [11]:
SVM_scores(x_train,x_dev,x_test,y_train1,y_dev1,y_test1)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ),